In [11]:
from Bio import Entrez
import pandas as pd
import xml.etree.ElementTree as ET
import time

In [12]:
Entrez.email = "leilaniarvizu4@gmail.com"
Entrez.api_key = 'c3059affa8a562390b817e380d1f6bc8b308'

In [ ]:
# Biosample ID
biosample_id = "biosample id"

# Fetch the Biosample data in XML format
handle = Entrez.efetch(db="biosample", id=biosample_id, rettype="xml", retmode="text") 
xml_data = handle.read()
handle.close()

root = ET.fromstring(xml_data) #xml to string conversion


In [14]:
#able to extract information from biosample database 
attributes = root.findall(".//Attributes/Attribute")
geo_dict = {} #dict to later make into a dataframe
if attributes:
    for attribute in attributes:
        attribute_name = attribute.attrib.get("attribute_name") #getting the name of attribute 
        #print(attribute_name)
        attribute_value = attribute.text # the value of the atrribute 
        #print(attribute_value)
        if attribute_name == 'geo_loc_name': #getting the location of strain
            geo_dict['Sample ID'] = biosample_id
            geo_dict['Geographic Location'] = attribute_value
            continue
        elif attribute_name == 'isolation_source': #getting information of strain isolation 
            geo_dict['Isolation Source'] = attribute_value
            continue 
        elif attribute_name == 'strain': #getting strain name of bacteria
            geo_dict['Strain'] = attribute_value

    # adding None if information is not avalialbe 
    if 'Sample ID' not in geo_dict:
        geo_dict['Sample ID'] = biosample_id   
    if 'Geographic Location' not in geo_dict:
        geo_dict['Geographic Location'] = "None"
    if 'Isolation Source' not in geo_dict:
        geo_dict['Isolation Source'] = 'None'
    if 'Strain' not in geo_dict:
        geo_dict['Strain'] = 'None'
        

#geo_dict


In [15]:
#open the biosample id 
bio_df = pd.read_csv('../biosample analysis/data_summary.tsv', sep='\t',encoding='utf-8')
biosample_id_list = bio_df['BioSample'].tolist() #list of all ~600 bacteria strains
#biosample_id_list 
len(biosample_id_list)

692

In [16]:
#creating a function to get attributes 
def attribute(biosample_id): #must be BioSample ID

    xml_data = False
    while xml_data is False:
        try:
            #wait so that we don't get an error from ncbi 
            time.sleep(1)
            
            # Fetch the Biosample data in XML format
            handle = Entrez.efetch(db="biosample", id=biosample_id, rettype="xml", retmode="text") 
            xml_data = handle.read()
            handle.close()
        except:
            print('error with', biosample_id)

    root = ET.fromstring(xml_data) #xml to string conversion
    attributes = root.findall(".//Attributes/Attribute")
    
    return(attributes) #give you the attrbutes assoitated with the strain

In [ ]:
attr = attribute('biosample id')


In [7]:
#creating function to create geo_dict 

def geo(attributes, biosample_id): #we need the attributes for the strain and the strain's biosample id
    geo_dict = {} #dict to later make into a dataframe
    if attributes:
        for attribute in attributes:
            attribute_name = attribute.attrib.get("attribute_name") #getting the name of attribute 
            #print(attribute_name)
            attribute_value = attribute.text # the value of the atrribute 
            #print(attribute_value)
            if attribute_name == 'geo_loc_name': #getting the location of strain
                geo_dict['Sample ID'] = biosample_id
                geo_dict['Geographic Location'] = attribute_value
                continue
            elif attribute_name == 'isolation_source': #getting information of strain isolation 
                geo_dict['Isolation Source'] = attribute_value
                continue 
            elif attribute_name == 'strain': #getting strain name of bacteria
                geo_dict['Strain'] = attribute_value
    # adding None if information is not avaliable 
    if 'Sample ID' not in geo_dict:
        geo_dict['Sample ID'] = biosample_id   
    if 'Geographic Location' not in geo_dict:
        geo_dict['Geographic Location'] = "None"
    if 'Isolation Source' not in geo_dict:
        geo_dict['Isolation Source'] = 'None'
    if 'Strain' not in geo_dict:
        geo_dict['Strain'] = 'None'
    
    return(geo_dict)


In [ ]:
geo(attr,'biosample id')

{'Strain': 'KGMB02009',
 'Sample ID': 'SAMN18309827',
 'Geographic Location': 'South Korea:Gyeonggi-do, Seongnam-si',
 'Isolation Source': 'None'}

In [ ]:
# for loop to generate dictionaries for each ID
# start with an empty dataframe with keys that match for columns
# geo_loc_df = pd.DataFrame()

for bio_id in biosample_id_list:
    # Generate a dictionary for the current ID
    attr = attribute(bio_id)
    geo_loc = geo(attr, bio_id)
    # add row to dataframe based on keys for geo_loc
    print(geo_loc)